In [5]:
!pip install satx
import tensorflow as tf
import tensorflow_datasets as tfds
import satx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for satx: filename=SATX-0.3.8-py3-none-any.whl size=19760 sha256=19ef2901b8444c8986d2fbf92087a140e29ed21f5b28a75d8970e55f87558633
  Stored in directory: /root/.cache/pip/wheels/e4/fe/74/0e3f5376ea2f163471974f935a8f86989b4aab641aa6a8567c
Successfully built satx


In [6]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [7]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [8]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [13]:
def oracle(seq): 

    units = sum([1 << i for i, bit in enumerate(seq) if bit == 1])

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(units=units, activation=tf.keras.activations.relu),)
    model.add(tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    model.fit(ds_train, epochs=1)
    
    result = model.evaluate(ds_test)

    del model

    print('ORACLE: {} - {}'.format(result, units))

    return result[0] # loss

In [14]:
seq = satx.hess_binary(n=10, oracle=oracle)
print(seq)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


79/79 [==============================] - 0s 2ms/step - loss: 2.0215 - sparse_categorical_accuracy: 0.2301
ORACLE: [2.0215306282043457, 0.23010000586509705] - 1
79/79 [==============================] - 0s 2ms/step - loss: 1.4003 - sparse_categorical_accuracy: 0.5499
ORACLE: [1.4002909660339355, 0.5498999953269958] - 3
79/79 [==============================] - 0s 2ms/step - loss: 0.4994 - sparse_categorical_accuracy: 0.8602
ORACLE: [0.4994032680988312, 0.8601999878883362] - 7
79/79 [==============================] - 0s 2ms/step - loss: 0.3280 - sparse_categorical_accuracy: 0.9108
ORACLE: [0.3279992341995239, 0.9107999801635742] - 15
79/79 [==============================] - 0s 2ms/step - loss: 0.2828 - sparse_categorical_accuracy: 0.9201
ORACLE: [0.28281930088996887, 0.9200999736785889] - 31
79/79 [==============================] - 0s 3ms/step - loss: 0.2261 - sparse_categorical_accuracy: 0.9378
ORACLE: [0.2260565608739853, 0.9377999901771545] - 63
79/79 [==============================] - 

In [15]:
units = sum([1 << i for i, bit in enumerate(seq) if bit == 1])

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(units=units, activation=tf.keras.activations.relu),)
model.add(tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax))

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(ds_train, epochs=10)

result = model.evaluate(ds_test)

Epoch 1/10
469/469 [==============================] - 12s 22ms/step - loss: 0.2326 - sparse_categorical_accuracy: 0.9341
Epoch 2/10
469/469 [==============================] - 9s 18ms/step - loss: 0.0887 - sparse_categorical_accuracy: 0.9739
Epoch 3/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0565 - sparse_categorical_accuracy: 0.9831
Epoch 4/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0385 - sparse_categorical_accuracy: 0.9884
Epoch 5/10
469/469 [==============================] - 8s 16ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9922
Epoch 6/10
469/469 [==============================] - 8s 17ms/step - loss: 0.0194 - sparse_categorical_accuracy: 0.9944
Epoch 7/10
469/469 [==============================] - 8s 18ms/step - loss: 0.0142 - sparse_categorical_accuracy: 0.9958
Epoch 8/10
469/469 [==============================] - 8s 18ms/step - loss: 0.0097 - sparse_categorical_accuracy: 0.9979
Epoch 9/10
469/469 [===================

In [16]:
result

[0.0693831592798233, 0.9810000061988831]

In [17]:
units

1007